By Mike Silva

## Introduction

The goal of this project is to practice beginning to work with a distributed recommender system.  For this project I will be using data scrapped from BoardGameGeek.com (BGG).

### About the BGG Dataset
The BoardGameGeek dataset was collected by myself by scrapping data from the API that forms the backend of [BoardGameGeek's website](https://boardgamegeek.com/). Data scrapping in ongoing but this particular data set has over 1.9 million ratings (implicit and explicit) for about 88,000 games by 219,000 users. I have previously exported the ratings from the SQLite database and uploaded them to the [Databricks Community Platform](https://community.cloud.databricks.com/).

In [2]:
# Load the ratings data into a data frame
df = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load("/FileStore/tables/bgg_ratings.csv")
# Create a view
df.createOrReplaceTempView("ratings")
# Show the data
df.show()

+---+-------+-------+------+-------------------+
 id|item_id|user_id|rating| rating_tstamp|
+---+-------+-------+------+-------------------+
 1| 3| 987| 9.0|2016-07-29 08:22:03|
 2| 3| 940| 9.0|2015-05-19 09:11:04|
 3| 3| 607| 8.0|2002-08-25 00:49:42|
 4| 3| 8972| 8.0|2004-02-28 22:32:59|
 5| 3| 8973| 8.0|2005-10-21 12:51:52|
 6| 3| 8974| 8.0|2008-01-01 10:30:13|
 7| 3| 960| 8.0|2006-10-03 01:36:15|
 8| 3| 948| 8.0|2011-09-20 17:47:38|
 9| 3| 936| 8.0|2009-01-04 04:31:36|
 10| 3| 8975| 8.0|2013-06-19 12:17:14|
 11| 3| 1499| 7.0|2003-08-19 04:44:51|
 12| 3| 8976| 7.0|2002-06-28 05:14:01|
 13| 3| 181| 7.0|2019-01-18 05:07:16|
 14| 3| 8977| 7.0|2008-01-20 11:36:11|
 15| 3| 8978| 7.0|2013-08-11 10:04:44|
 16| 3| 127| 7.0|2015-04-06 05:20:14|
 17| 3| 4754| 6.8|2009-04-05 10:43:06|
 18| 3| 8979| 6.5|2007-06-11 04:30:38|
 19| 3| 8980| 6.5|2005-07-24 09:57:20|
 20| 3| 8981| 6.5|2006-04-20 22:44:14|
+---+-------+-------+------+-------------------+
only showing top 20 rows

## Data Exploration & Preparation

Now that I have the data loaded let's quickly explore the data prior to spliting it into training and test sets.  Let's see how many levels of ratings exists in this data set. First we'll look at the descriptive statistics then the counts per rating.

In [4]:
df.describe("rating").show()

+-------+------------------+
summary| rating|
+-------+------------------+
 count| 1936359|
 mean|3.9819450749938494|
 stddev| 4.058472760594837|
 min| 0.0|
 max| 10.0|
+-------+------------------+

In [5]:
%sql
SELECT `rating`, SUM(1) AS `count` FROM `ratings` GROUP BY rating ORDER BY rating ASC;

rating,count
0.0,923033
1.0E-38,1
0.01,1
0.1,1
0.666,1
1.0,12198
1.00002,1
1.01,4
1.06,2
1.1,23


We see that these geeks have a sense of humor and give ratings on a continuous scale from zero to ten.  I do know that some of the zeros are implicit ratings.  The first thing I will do is round the data so that it is to the nearest integer ranging from zero to 10.

In [7]:
import pyspark.sql.functions as func
from pyspark.sql.types import IntegerType

df = df.withColumn("rounded_rating", func.round(df["rating"], 0).cast(IntegerType()))
df.createOrReplaceTempView("ratings")
df.show()

+---+-------+-------+------+-------------------+--------------+
 id|item_id|user_id|rating| rating_tstamp|rounded_rating|
+---+-------+-------+------+-------------------+--------------+
 1| 3| 987| 9.0|2016-07-29 08:22:03| 9|
 2| 3| 940| 9.0|2015-05-19 09:11:04| 9|
 3| 3| 607| 8.0|2002-08-25 00:49:42| 8|
 4| 3| 8972| 8.0|2004-02-28 22:32:59| 8|
 5| 3| 8973| 8.0|2005-10-21 12:51:52| 8|
 6| 3| 8974| 8.0|2008-01-01 10:30:13| 8|
 7| 3| 960| 8.0|2006-10-03 01:36:15| 8|
 8| 3| 948| 8.0|2011-09-20 17:47:38| 8|
 9| 3| 936| 8.0|2009-01-04 04:31:36| 8|
 10| 3| 8975| 8.0|2013-06-19 12:17:14| 8|
 11| 3| 1499| 7.0|2003-08-19 04:44:51| 7|
 12| 3| 8976| 7.0|2002-06-28 05:14:01| 7|
 13| 3| 181| 7.0|2019-01-18 05:07:16| 7|
 14| 3| 8977| 7.0|2008-01-20 11:36:11| 7|
 15| 3| 8978| 7.0|2013-08-11 10:04:44| 7|
 16| 3| 127| 7.0|2015-04-06 05:20:14| 7|
 17| 3| 4754| 6.8|2009-04-05 10:43:06| 7|
 18| 3| 8979| 6.5|2007-06-11 04:30:38| 7|
 19| 3| 8980| 6.5|2005-07-24 09:57:20| 7|
 20| 3| 8981| 6.5|2006-04-20 22:44:14| 7|
+---+-------+-------+------+-------------------+--------------+
only showing top 20 rows

In [8]:
%sql
SELECT rounded_rating, SUM(1) AS count FROM ratings GROUP BY rounded_rating ORDER BY rounded_rating ASC;

rounded_rating,count
0,923036
1,12283
2,9950
3,17967
4,30799
5,59759
6,111315
7,178465
8,216520
9,169258


Let's see if all of the zero ratings are implicit or if some are explicit.  All implicit ratings have a zero rating and no timestamp.

In [10]:
%sql
SELECT rounded_rating, SUM(1) AS count FROM ratings WHERE  rating_tstamp IS NOT NULL OR rounded_rating > 0 GROUP BY rounded_rating ORDER BY rounded_rating ASC;

rounded_rating,count
0,3
1,12283
2,9950
3,17967
4,30799
5,59759
6,111315
7,178465
8,216520
9,169258


Generally the ratings are quite positive.  Games receiving an 8 or higher will be the target of the recommendation system as a "good" recommendation.

### Extract Explicit Ratings

The recommender system will be based off of the explicit ratings.  I will split them off into a seperate data set.

In [12]:
explicit_rating = df.filter("rating_tstamp IS NOT NULL OR rounded_rating > 0")
explicit_rating.createOrReplaceTempView("explicit_rating")
explicit_rating.describe("rounded_rating").show()

+-------+------------------+
summary| rounded_rating|
+-------+------------------+
 count| 1013326|
 mean| 7.648843511367517|
 stddev|1.9708746678861617|
 min| 0|
 max| 10|
+-------+------------------+

I want to build the recommender system off of detailed data.  I will require that a user must have given 25 or more ratings, and a game will need to have 10 or more ratings to be included in the data set.  I will do these filters sequentially.

In [14]:
user_filter = explicit_rating.groupBy("user_id").count().filter("count >= 25").select("user_id").withColumnRenamed("user_id", "filter_user_id")
explicit_rating = explicit_rating.join(user_filter, explicit_rating.user_id == user_filter.filter_user_id)
explicit_rating.createOrReplaceTempView("explicit_rating")
item_filter = explicit_rating.groupBy("item_id").count().filter("count >= 10").select("item_id").withColumnRenamed("item_id", "filter_item_id")
explicit_rating = explicit_rating.join(item_filter, explicit_rating.item_id == item_filter.filter_item_id).drop(*["filter_user_id", "filter_item_id"])
explicit_rating.createOrReplaceTempView("explicit_rating")
explicit_rating.describe("rounded_rating").show()

+-------+------------------+
summary| rounded_rating|
+-------+------------------+
 count| 369017|
 mean| 7.723828441508115|
 stddev|1.7776787046235145|
 min| 1|
 max| 10|
+-------+------------------+

### Train Test Split

There is just under 370,000 ratings left.  Now I will split the data into training and test sets.  I will train the models on 80% of the data and use the remainder to test the accuracy of the recommender system.

In [16]:
seed = 42
fractions = explicit_rating.select("rounded_rating").distinct().withColumn("fraction", func.lit(0.8)).rdd.collectAsMap()
train = explicit_rating.stat.sampleBy("rounded_rating", fractions, seed)
train.createOrReplaceTempView("train")
train.describe("rounded_rating").show()

+-------+-----------------+
summary| rounded_rating|
+-------+-----------------+
 count| 295054|
 mean|7.726595809580619|
 stddev| 1.77683438649093|
 min| 1|
 max| 10|
+-------+-----------------+

In [17]:
test = explicit_rating.subtract(train)
test.createOrReplaceTempView("test")
test.describe("rounded_rating").show()

+-------+------------------+
summary| rounded_rating|
+-------+------------------+
 count| 73963|
 mean|7.7127888268458555|
 stddev|1.7810121219349713|
 min| 1|
 max| 10|
+-------+------------------+